# Transformers Pipeline Work

In [2]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [3]:
# 同庄老师课堂所说
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import confusion_matrix
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import MultiStepLR, CosineAnnealingLR
from torch.nn import Module, Linear, Dropout, LayerNorm, Identity
import pandas as pd
import numpy as np
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import time
from torch.autograd import Variable
use_cuda = torch.cuda.is_available()
print('okk')

okk


In [4]:
# *------------------- CCT: Transformer -------------------------*
# Meta CNN, CCT transformer
class Attention(Module):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    """

    def __init__(self, dim, num_heads=8, attention_dropout=0.1, projection_dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // self.num_heads
        self.scale = head_dim ** -0.5

        self.qkv = Linear(dim, dim * 3, bias=False)
        self.attn_drop = Dropout(attention_dropout)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(projection_dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # 3, Batch , num_heads , N(seq_len) , channels // num_heads (scale)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class TransformerEncoderLayer(Module):
    """
    Inspired by torch.nn.TransformerEncoderLayer and
    rwightman's timm package.
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 attention_dropout=0.1, drop_path_rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.pre_norm = LayerNorm(d_model)
        self.self_attn = Attention(dim=d_model, num_heads=nhead,
                                   attention_dropout=attention_dropout, projection_dropout=dropout)

        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNorm(d_model)
        self.linear2 = Linear(dim_feedforward, d_model)
        self.dropout2 = Dropout(dropout)

        self.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0 else Identity()

        self.activation = F.gelu

    def forward(self, src: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        src = src + self.drop_path(self.self_attn(self.pre_norm(src)))
        src = self.norm1(src)
        src2 = self.linear2(self.dropout1(self.activation(self.linear1(src))))
        src = src + self.drop_path(self.dropout2(src2))
        return src


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(Module):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Tokenizer(nn.Module):
    def __init__(self,
                 kernel_size, stride, padding,
                 pooling_kernel_size=3, pooling_stride=2, pooling_padding=1,
                 n_conv_layers=1,
                 n_input_channels=3,
                 n_output_channels=64,
                 in_planes=64,
                 activation=None,
                 max_pool=True):
        super(Tokenizer, self).__init__()
        # 不用麻烦了，就搞一层
        n_filter_list = [n_input_channels] + \
                        [in_planes for _ in range(n_conv_layers - 1)] + \
                        [n_output_channels]
        # input -- 中间过渡 in_planes * in_planes -- output channel

        self.conv_layers = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(n_filter_list[i], n_filter_list[i + 1],
                          kernel_size=(kernel_size, kernel_size),
                          stride=(stride, stride),
                          padding=(padding, padding), bias=False),
                nn.Identity() if activation is None else activation(),
                nn.MaxPool2d(kernel_size=pooling_kernel_size,
                             stride=pooling_stride,
                             padding=pooling_padding) if max_pool else nn.Identity()
            )
                for i in range(n_conv_layers)
            ])

        self.flattener = nn.Flatten(2, 3)
        self.apply(self.init_weight)

    def sequence_length(self, n_channels=3, height=32, width=32):
        return self.forward(torch.zeros((1, n_channels, height, width))).shape[1]  # 你这，有点不优异，自己又跑了一遍可还行

    def forward(self, x):
        # return self.flattener(self.conv_layers(x)).transpose(-2, -1)
        out = self.conv_layers(x)  # 连着对 H、W downsample两次， 砍成1/4
        return self.flattener(out).transpose(-2, -1)  # [BS , downsample(H*W) , output C]

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)


class TransformerClassifier(nn.Module):
    def __init__(self,
                 seq_pool=True,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout_rate=0.1,
                 attention_dropout=0.1,
                 stochastic_depth_rate=0.1,
                 positional_embedding='sine',
                 sequence_length=None,
                 *args, **kwargs):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.seq_pool = seq_pool

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        if not seq_pool:
            sequence_length += 1
            self.class_emb = nn.Parameter(torch.zeros(1, 1, self.embedding_dim),
                                          requires_grad=True)
        else:
            self.attention_pool = nn.Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = nn.Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                   requires_grad=True)
                nn.init.trunc_normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = nn.Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                   requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = nn.Dropout(p=dropout_rate)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth_rate, num_layers)]
        self.blocks = nn.ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout_rate,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])  # 几层 transformer
        self.norm = nn.LayerNorm(embedding_dim)

        self.fc = nn.Linear(embedding_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if not self.seq_pool:
            cls_token = self.class_emb.expand(x.shape[0], -1, -1)
            x = torch.cat((cls_token, x), dim=1)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)
        # transformers 堆叠
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        if self.seq_pool:
            x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)
        else:
            x = x[:, 0]

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)


class CCT(nn.Module):
    def __init__(self,
                 img_size=32,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=3,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 *args, **kwargs):
        super(CCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,  # 起码三件套得整一手吧
                                   n_conv_layers=n_conv_layers)

        self.classifier = TransformerClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            seq_pool=True,
            dropout_rate=0.,
            attention_dropout=0.1,
            stochastic_depth=0.1,
            *args, **kwargs)

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)
print('okk')

okk


In [5]:
def train(model, train_X, loss_func, optimizer):
    model.train()
    total_loss = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    for i, (x, y) in enumerate(train_X):
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        loss = loss_func(outputs, y)
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.shape[0]
        if (i + 1) % 300 == 0:
            print("Step [{}/{}] Train Loss: {:.4f}".format(i + 1, len(train_X), loss.item()))

    print(total_loss / len(train_X.dataset))  # 其实loss计算有点小问题
    return total_loss / len(train_X.dataset)


def predict(model,test_X):
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    top1_acc = top5_acc = 0
    with torch.no_grad():
        for i , (x,y) in enumerate(test_X):
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, indice = output.topk(5, -1, True, sorted= True)
            top1_acc += torch.eq(indice[:, 0], y).sum().item()
            top5_acc += torch.eq(indice , y.view(-1,1)).sum().item()
    return top1_acc/len(test_X.dataset) , top5_acc/len(test_X.dataset)
print('okk')

okk


# mixup

In [6]:
mixup=True
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# dataLoader


In [7]:
def get_loader(batch_size=128,num_workers=4,mixup=True,alpha=1):
    collator = torch.utils.data.dataloader.default_collate

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_dataset = datasets.CIFAR10(root='cifar', train=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize,]), download=True)
      
    dev_dataset=datasets.CIFAR10(root='cifar', train=False, transform=transforms.Compose([                                                                    
                transforms.ToTensor(),
                normalize,]))

    train_dataloader = torch.utils.data.DataLoader(train_dataset,      
                       batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, pin_memory=True,
                       drop_last=True,collate_fn=collator)
    
    dev_dataloader = torch.utils.data.DataLoader( dev_dataset, 
                     batch_size=batch_size, shuffle=False,
                     num_workers=num_workers, pin_memory=True,
                     drop_last=False)
    return train_dataloader,dev_dataloader

In [8]:
train_dataloader,dev_dataloader=get_loader(batch_size=128,num_workers=4,mixup=True,alpha=1)


Extracting cifar/cifar-10-python.tar.gz to cifar


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
date = '66R2'
if mixup:
  signal = 'CCT7_MIXUP_' + date + '.pth'
else:
  signal = 'CCT7_' + date + '.pth'

PATH = '/content/mydrive/MyDrive/pj2/outputs/' + signal
print(PATH)

/content/mydrive/MyDrive/pj2/outputs/CCT7_MIXUP_66R2.pth


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CCT(num_layers= 7,
        num_heads= 4,
        mlp_ratio= 2,
        embedding_dim= 256,
        kernel_size=3)
model.to(device)
# copy from github
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay= 5e-4,nesterov=True)
# scheduler = MultiStepLR(optimizer, milestones=[60,120,160,180], gamma=0.5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,200,0)
# scheduler = CosineAnnealingLR(optimizer, 200, 0)

loss_func = nn.CrossEntropyLoss()
# scheduler = MultiStepLR(optimizer, milestones=[30,60,90,120,150,160,180], gamma=0.7)
print('refresh')
best_accuracy = None

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


refresh


In [11]:
def adjust_learning_rate(optimizer, epoch ):
    import math
    lr = 1e-3
    Epochs = 200
    warmup = 5

    if epoch < warmup:
        lr = lr / (warmup - epoch)
    else:
        lr *= 0.5 * (1. + math.cos(math.pi * (epoch - warmup) / (Epochs - warmup)))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
print('okk')

okk


# train

In [ ]:
import time
start = time.time()
Iter_Time = 200
loss_train = np.zeros(Iter_Time)
test_acc= np.zeros(Iter_Time)
test_5acc= np.zeros(Iter_Time)
train_acc = np.zeros(Iter_Time)
print('start to train' + '*'*20)
if best_accuracy is None:
  best_accuracy = 0   
for epoch in range(Iter_Time):
    adjust_learning_rate(optimizer , epoch)
    loss = train(model= model, train_X=train_dataloader, loss_func=loss_func,optimizer=optimizer)
    loss_train[epoch] = loss
    top1 , top5 = predict(model=model, test_X =dev_dataloader)
    test_acc[epoch] = top1
    test_5acc[epoch] = top5
    print('训练轮次',epoch, 'loss:' , loss ,'top1 & top5:' ,top1 , top5 , '总训练时间:', time.time() - start)
   

    if best_accuracy < top1:
      best_accuracy = top1 
      torch.save(model,'/content/mydrive/MyDrive/pj2/outputs/epoch_%d_accuracy_%f'%(epoch,top1))
      print('Model is saved in /content/mydrive/MyDrive/pj2/outputs/epoch_%d_accuracy_%f'%(epoch,top1))
    if top1 < 0.15: break
    




start to train********************


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Step [300/390] Train Loss: 2.4002
3.556126202392578
训练轮次 0 loss: 3.556126202392578 top1 & top5: 0.2241 0.7233 总训练时间: 36.80139970779419
Model is saved in /content/mydrive/MyDrive/pj2/outputs/epoch_0_accuracy_0.224100
Step [300/390] Train Loss: 2.2201
2.23894413269043
训练轮次 1 loss: 2.23894413269043 top1 & top5: 0.2468 0.7595 总训练时间: 74.2878987789154
Model is saved in /content/mydrive/MyDrive/pj2/outputs/epoch_1_accuracy_0.246800
Step [300/390] Train Loss: 2.1808
2.1311169262695313
训练轮次 2 loss: 2.1311169262695313 top1 & top5: 0.2638 0.7727 总训练时间: 111.03606700897217
Model is saved in /content/mydrive/MyDrive/pj2/outputs/epoch_2_accuracy_0.263800
Step [300/390] Train Loss: 2.0967
2.0505310052490233
训练轮次 3 loss: 2.0505310052490233 top1 & top5: 0.2889 0.8018 总训练时间: 147.6942183971405
Model is saved in /content/mydrive/MyDrive/pj2/outputs/epoch_3_accuracy_0.288900
Step [300/390] Train Loss: 1.9535
1.9599160586547852
训练轮次 4 loss: 1.9599160586547852 top1 & top5: 0.3044 0.8361 总训练时间: 184.36888647079

In [ ]:
trainingloss=pd.DataFrame(data=loss_train)
trainingloss.to_csv('/content/mydrive/MyDrive/pj2/outputs/exp3/trainingloss.csv',encoding='utf-8')

trainingacc=pd.DataFrame(data=train_acc)
trainingacc.to_csv('/content/mydrive/MyDrive/pj2/outputs/exp3/trainingacc.csv',encoding='utf-8')


valloss=pd.DataFrame(data=valloss)
valloss.to_csv('/content/mydrive/MyDrive/pj2/outputs/exp3/valloss.csv',encoding='utf-8')

valacc=pd.DataFrame(data=test_acc)
valacc.to_csv('/content/mydrive/MyDrive/pj2/outputs/exp3/testacc.csv',encoding='utf-8')

valacc5=pd.DataFrame(data=test_5acc)
valacc5.to_csv('/content/mydrive/MyDrive/pj2/outputs/exp3/valacc5.csv',encoding='utf-8')